# Estimating Motors from a Variety of Geometric Data in 3D Conformal Geometric Algebra

Robert Valkenburg and Leo Dorst

In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=2, suppress=True)
import versor as vsr
from versor.drawing import *
from game import (MotorEstimationSolver, VDMotorEstimationSolver)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import versor as vsr

## Dataset Generation

In [74]:
delta = 9e6
theta = np.pi/6.0
motor = (vsr.Vec(1,1,1).unit() * delta).trs() * vsr.Rot(vsr.Biv(1,2,3).unit() * theta )
print(motor)

Mot: [ 0.87 -0.13 -0.27 -0.4 -3.3e+06 -2.9e+06 -5.1e+05 6.9e+05 ]


In [75]:
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.rand(3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [76]:
lines_a = [vsr.Dll(vsr.Vec(*np.random.normal(0.0,0.8,3)).null(),
                   vsr.Vec(*np.random.normal(0.0,0.8,3)).null())
          for i in range(10)]
lines_b = [line.spin(motor) for line in lines_a]

In [77]:
n_planes = 100
planes_a = []
for i in range(10):
    dir_vec = vsr.Vec(*np.random.normal(0.0, 0.8, 3)).unit()
    distance = np.random.normal(0.0, 0.8,1)
    planes_a.append(vsr.Dlp(dir_vec[0],dir_vec[1],dir_vec[2],distance[0]))
planes_b = [plane.spin(motor) for plane in planes_a]

In [78]:
n_drvs = 10
drvs_a = [vsr.Vec(*np.random.rand(3)).unit().drv() for i in range(10)]
drvs_b = [d.spin(motor) for d in drvs_a]

In [79]:
vd = VDMotorEstimationSolver()
# vd.add_dual_plane_observations(planes_a, planes_b)
# vd.add_dual_line_observations(lines_a, lines_b)
vd.add_point_observations(points_a, points_b)
vd_estimated_motor = vd.solve()
print(np.array(vd_estimated_motor))
print(np.array(motor))
print(motor * vd_estimated_motor.rev())

[       0.87       -0.15       -0.27       -0.39 -3335905.1  -2888695.24
  -532869.76   702301.04]
[       0.87       -0.13       -0.27       -0.4  -3291547.61 -2944365.07
  -514087.31   694365.07]
Mot: [ 1 0.0087 0.0084 -0.011 8.9e+04 -1e+05 1.4e+04 -3.5e-10 ]


In [10]:
for a, b in zip(points_a, points_b):
    print(b - a)

Dlp: [ -0.2935 1 1.283 2.802 ]
Dlp: [ 0.3598 1 1.251 2.335 ]
Dlp: [ 0.5233 1 1.398 2.239 ]
Dlp: [ -0.2367 1 1.41 2.865 ]
Dlp: [ 0.7605 1 0.9634 2.182 ]
Dlp: [ 0.4225 1 1.696 2.779 ]
Dlp: [ 0.6947 1 0.8519 1.961 ]
Dlp: [ 0.4056 1 1.495 2.456 ]
Dlp: [ 0.3746 1 1.345 2.206 ]
Dlp: [ 0.382 1 1.451 2.578 ]


In [11]:
scene = Scene(children=[point_mesh(points_a[0],color='hotpink'),
                        point_mesh(points_b[0],color='hotpink'),
#                         plane_mesh(points_b[0] - points_a[0], width=10, height=10,color='lightskyblue'),
                       frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [12]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
# for a,b in zip(drvs_a, drvs_b):
#     mes.add_direction_vector_inner_product_residual_block(a,b)
for a,b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(a,b)
#     mes.add_line_commutator_residual_block(a,b) 
#     mes.add_line_inner_product_residual_block(a,b)
#     mes.add_line_angle_distance_norm_residual_block(a,b)
    
# for a,b in zip(points_a, points_b):
#     mes.add_point_distance_residual_block(a,b)
#     mes.add_point_correspondences_residual_block(a,b)
# for a, b, c, d, e, f in zip(points_a, points_b, lines_a, lines_b, planes_a, planes_b):
#     mes.add_point_difference_residual_block(a,b)
#     mes.add_line_correspondences_residual_block(c,d)
#     mes.add_dual_plane_difference_residual_block(e,f)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
# mes.num_linear_solver_threads = 4
# mes.num_threads = 4
# mes.function_tolerance  = 1e-16

mes.max_num_iterations = 1000
# mes.minimizer_type = 'LINE_SEARCH'

te_estimated_motor, summary, motors = mes.solve()
print(summary['full_report'])
print(np.array(te_estimated_motor))


Solver Summary (v 1.12.0-eigen-(3.3.1)-lapack-suitesparse-(4.5.3)-cxsparse-(3.1.9)-no_openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            10                       10
Residual                                   60                       60

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1
Linear solver ordering              AUTOMATIC        

In [13]:
print(np.array(motor))

[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]


In [14]:
np.allclose(vd_estimated_motor, te_estimated_motor, atol=1e-03)

False

In [15]:
dlp = vsr.Dlp(1,1,1,5)
Jac = np.zeros((5,8))
for i in range(8):
    ei = vsr.Mot(0,0,0,0,0,0,0,0)
    ei[i] = 1.
    Jac[:,i] = np.array((vsr.CGA(motor) * vsr.CGA(dlp) * vsr.CGA(ei.rev()) +
                         vsr.CGA(ei) * vsr.CGA(dlp) * vsr.CGA(motor.rev())))[1:6]
Jac = np.delete(Jac,3,0)
print(Jac)

[[  0.73   1.73   2.73   1.     0.     0.     0.     0.  ]
 [  1.73  -0.73  -1.     2.73   0.     0.     0.     0.  ]
 [  2.73   1.    -0.73  -1.73   0.     0.     0.     0.  ]
 [ 11.26   1.    -4.5    1.    -0.73  -1.73  -2.73  -1.  ]]


In [16]:
Jac3 = np.zeros((8,6))
Jac3[:,0] = vsr.Dll(1,0,0,0,0,0) * motor
Jac3[:,1] = vsr.Dll(0,1,0,0,0,0) * motor
Jac3[:,2] = vsr.Dll(0,0,1,0,0,0) * motor
Jac3[:,3] = vsr.Dll(0,0,0,1,0,0) * motor
Jac3[:,4] = vsr.Dll(0,0,0,0,1,0) * motor
Jac3[:,5] = vsr.Dll(0,0,0,0,0,1) * motor
print(Jac3)

[[ 0.    0.5   0.    0.    0.    0.  ]
 [ 0.87  0.   -0.5   0.    0.    0.  ]
 [ 0.    0.87  0.    0.    0.    0.  ]
 [ 0.5   0.    0.87  0.    0.    0.  ]
 [-0.43 -0.18  0.25  0.87  0.    0.5 ]
 [ 0.68 -0.25 -0.18  0.    0.87  0.  ]
 [ 0.25  0.68  0.43 -0.5   0.    0.87]
 [-0.18  0.43 -0.68  0.    0.5   0.  ]]


In [17]:
np.dot(Jac,Jac3)

array([[ 2.  ,  2.73,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.73, -0.  ,  2.73,  0.  ,  0.  ,  0.  ],
       [-0.  ,  0.73, -2.  ,  0.  ,  0.  ,  0.  ],
       [-0.  ,  0.  ,  0.  ,  0.73, -2.  , -2.73]])

In [18]:
p1s = [vsr.Vec(1,0,0).null(), vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,1).null()] 
p2s = [p1i.spin(motor) for p1i in p1s] 
p3s = [(p2i.vec() - p1i.vec()).null() for p1i, p2i in zip(p1s,p2s)]
c1 = vsr.Cir(*p1s)
c2 = vsr.Cir(*p2s)
c3 = vsr.Cir(*p3s)

In [19]:
np.array((vsr.CGA(motor) * vsr.CGA(c1.axis()) * vsr.CGA(vsr.Mot(1,0,0,0,0,0,0,0).rev()) +
 vsr.CGA(vsr.Mot(1,0,0,0,0,0,0,0)) * vsr.CGA(c1.axis()) * vsr.CGA(motor.rev())))[6:16]

array([ 1.82, -0.71, -0.11,  0.  ,  0.  ,  0.  ,  0.41, -0.41,  0.41,  0.  ])

In [20]:
JacCircle = np.zeros((6,8))
for i in range(8):
    ei = vsr.Mot(0,0,0,0,0,0,0,0)
    ei[i] = 1.
    JacCircle[:,i] = np.delete(np.array((vsr.CGA(motor) * vsr.CGA(c1.axis()) * vsr.CGA(ei.rev()) +
                               vsr.CGA(ei) * vsr.CGA(c1.axis()) * vsr.CGA(motor.rev())))[6:16],[3,4,5,9])
print(JacCircle)

[[ 1.82  0.41  0.11 -0.71  0.    0.    0.    0.  ]
 [-0.71 -0.11  0.41 -1.82  0.    0.    0.    0.  ]
 [-0.11  0.71  1.82  0.41  0.    0.    0.    0.  ]
 [ 0.41 -0.41  0.41 -1.01  0.41 -1.82  0.71  0.11]
 [-0.41 -0.41  1.01  0.41  1.82  0.41  0.11 -0.71]
 [ 0.41 -1.01 -0.41  0.41 -0.71 -0.11  0.41 -1.82]]


In [21]:
np.dot(np.row_stack((JacCircle,JacCircle,JacCircle)),Jac3)

array([[-0.  ,  1.01, -0.82,  0.  ,  0.  ,  0.  ],
       [-1.01,  0.  , -1.52,  0.  ,  0.  ,  0.  ],
       [ 0.82,  1.52,  0.  ,  0.  ,  0.  ,  0.  ],
       [-2.12,  1.47,  0.  , -0.  , -1.52,  0.82],
       [-0.5 , -0.  ,  1.47,  1.52, -0.  ,  1.01],
       [ 0.  , -0.5 ,  2.12, -0.82, -1.01,  0.  ],
       [-0.  ,  1.01, -0.82,  0.  ,  0.  ,  0.  ],
       [-1.01,  0.  , -1.52,  0.  ,  0.  ,  0.  ],
       [ 0.82,  1.52,  0.  ,  0.  ,  0.  ,  0.  ],
       [-2.12,  1.47,  0.  , -0.  , -1.52,  0.82],
       [-0.5 , -0.  ,  1.47,  1.52, -0.  ,  1.01],
       [ 0.  , -0.5 ,  2.12, -0.82, -1.01,  0.  ],
       [-0.  ,  1.01, -0.82,  0.  ,  0.  ,  0.  ],
       [-1.01,  0.  , -1.52,  0.  ,  0.  ,  0.  ],
       [ 0.82,  1.52,  0.  ,  0.  ,  0.  ,  0.  ],
       [-2.12,  1.47,  0.  , -0.  , -1.52,  0.82],
       [-0.5 , -0.  ,  1.47,  1.52, -0.  ,  1.01],
       [ 0.  , -0.5 ,  2.12, -0.82, -1.01,  0.  ]])

In [22]:
trs = vsr.Vec(1,2,3).trs() * vsr.Biv(0,pi/6,0).exp()
print(trs)
cdiff = c1.spin(trs)

Mot: [ 0.87 0 -0.5 0 -1.2 -0.87 -1 -0.5 ]


In [23]:
ans = c1.dual().comm(cdiff.dual()).undual()
scene = Scene(children=[circle_mesh(c1,color='hotpink'),
                        point_mesh(c1.pnt(),color='hotpink'),
                        circle_mesh(cdiff,color='lightskyblue'),
                        point_mesh(cdiff.pnt(),color='lightskyblue'),
                        line_mesh(vsr.Dll(c1.pnt(), cdiff.pnt())),
                        bivector_mesh(cdiff.dir().unit()),
                        circle_mesh(ans,color='green'),
                        point_mesh(ans.pnt(),color='green'),
#                         vector_mesh(ans.dual().dir().vec(), arrow=True, position=(ans).pnt()),
                        frame_mesh(),
                        AmbientLight(color='#cccccc')] +\
             [point_mesh(p, color='red') for p in ans.dual().split()]
             )

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [24]:
c1.pnt()


Pnt: [ 0 0 0.5 1 0.125 ]

In [25]:
cdiff.pnt()

Pnt: [ 0.567 2 3.25 1 7.442 ]

In [26]:
ans.pnt()

Pnt: [ 0.04788 0.8923 1.848 1 2.108 ]

In [27]:
ans = c1.dual().comm(cdiff.dual())
ans

Par: [ 1.254 12.67 3.663 5.49 -7.5 -18.29 -5.826 -0.5849 13.19 40.25 ]

In [28]:
ans.radius()

1.4672999095746644

In [29]:
ans.dir().unit().unduale()

TypeError: Unable to convert function return value to a Python type! The signature was
	(self: versor_pybind11.Drv) -> vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)19, (short)21, (short)22> >

In [30]:
cdiff.dir().unit().unduale() 

Vec: [ 0.5 -0.41 -0.76 ]

In [31]:
c1.dir().unit().unduale() 

Vec: [ -0.41 -0.41 -0.82 ]

In [32]:
c1.radius()

1.118033988749895

In [33]:
cdiff.radius()

1.1180339887498945

In [34]:
c1.pnt().vec() - cdiff.pnt().vec()

Vec: [ -0.57 -2 -2.8 ]

In [35]:
(c1 - cdiff).dual().dir().vec()

Vec: [ 2.2 0 0.13 ]

In [36]:
ans[0]

TypeError: 'versor_pybind11.Par' object does not support indexing

In [37]:
- np.outer(l1,l2)ans[1].undual()

SyntaxError: invalid syntax (<ipython-input-37-3432a68e7cd6>, line 1)

In [38]:
vsr.Lin(c1 - cdiff).__print_debug_info_console()

In [39]:
c1.__print_debug_info_console()
cdiff.__print_debug_info_console()

In [40]:
dlp2 = dlp.spin(motor)
pd = dlp2 - dlp
scene = Scene(children=[
        vector_mesh(dlp.dir().vec(),color='hotpink', arrow=True),
        plane_mesh(dlp,color='hotpink'),
        plane_mesh(dlp2,color='lightskyblue'),
        plane_mesh(dlp2-dlp,color='red'),
                        frame_mesh(),
                        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

AttributeError: 'versor_pybind11.Mot' object has no attribute 'dir'

In [41]:
dlp2.dir()

Drv: [ -0.21 0.58 0.79 ]

In [42]:
dlp

Dlp: [ 1 1 1 5 ]

In [43]:
pd

Mot: [ 0 0 0 0 0 0 0 0 ]

In [44]:
l1 = lines_a[0]
l2 = lines_b[0]
Z = np.outer(l2,l1) - np.outer(l1,l2)
print(Z)

[[ 0.   -1.41  2.54  0.79  0.66 -4.39]
 [ 1.41  0.   -2.22  1.25 -2.52  5.59]
 [-2.54  2.22  0.   -3.49  3.5  -3.16]
 [-0.79 -1.25  3.49  0.    1.99 -7.01]
 [-0.66  2.52 -3.5  -1.99  0.    5.23]
 [ 4.39 -5.59  3.16  7.01 -5.23  0.  ]]


In [45]:
np.linalg.eig(Z)

(array([ 0.+13.7j,  0.-13.7j,  0. +0.j ,  0. -0.j , -0. +0.j , -0. -0.j ]),
 array([[ 0.08+0.27j,  0.08-0.27j,  0.65+0.j  ,  0.65-0.j  , -0.13-0.14j,
         -0.13+0.14j],
        [ 0.09-0.34j,  0.09+0.34j, -0.12-0.28j, -0.12+0.28j, -0.04-0.3j ,
         -0.04+0.3j ],
        [-0.29+0.19j, -0.29-0.19j, -0.28-0.2j , -0.28+0.2j ,  0.58+0.j  ,
          0.58-0.j  ],
        [ 0.02+0.42j,  0.02-0.42j, -0.31-0.36j, -0.31+0.36j, -0.31+0.22j,
         -0.31-0.22j],
        [-0.19-0.32j, -0.19+0.32j,  0.08-0.31j,  0.08+0.31j, -0.13+0.5j ,
         -0.13-0.5j ],
        [ 0.60+0.j  ,  0.60-0.j  , -0.17-0.14j, -0.17+0.14j,  0.27+0.21j,
          0.27-0.21j]]))

In [46]:
np.linalg.norm(l2.comm(l1) * 2.) / np.sqrt(2)

13.462110506950758

In [47]:
Z = np.zeros(6)
for a,b in zip(lines_a, lines_b):
    Z = Z + np.outer(a,b)

In [48]:
U,s,Vt = np.linalg.svd(Z)
np.dot(U.T,Vt)

array([[-0.53,  0.58, -0.55, -0.21, -0.13, -0.12],
       [-0.47, -0.57, -0.04, -0.37,  0.46, -0.33],
       [ 0.32,  0.27,  0.02,  0.23,  0.25, -0.84],
       [ 0.12, -0.03, -0.53,  0.44,  0.63,  0.33],
       [-0.45,  0.4 ,  0.63,  0.25,  0.39,  0.15],
       [-0.42, -0.33, -0.09,  0.71, -0.41, -0.19]])

In [49]:
np.array(motor)

array([ 0.87,  0.  , -0.5 ,  0.  , -0.68, -0.43, -0.18, -0.25])

In [50]:
n_circles = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.3, 3)).null() 
            for i in range(n_circles)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

radiuss = [np.random.rand() for i in range(n_circles)]
bivs = [vsr.Vec(*np.random.normal(0.0, 0.3, 3)).unit().unduale()
            for i in range(n_circles)]

circs_a = [vsr.Cir(p,r,b) for p,r,b in zip(points_a, radiuss, bivs)]
# circs_b = [vsr.Cir(p,r,b) for p,r,b in zip(points_b_noisy, radiuss, bivs)]
circs_b = [circle.spin(motor) for circle in circs_a]


In [51]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# initial_motor = vsr.Vec(2,3,4).trs() * vsr.Rot(vsr.Biv(0,-1,0) * np.pi/6)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(circs_a, circs_b):
    a1,a2 = a.dual().split()
    b1,b2 = b.dual().split()
    mes.add_line_correspondences_residual_block(vsr.Dll(a1,a2), vsr.Dll(b1,b2))
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary, motors) = mes.solve()
# print(summary['full_report'])
print(estimated_motor)

Mot: [ 0.87 -5e-14 -0.5 7e-14 -0.68 -0.43 -0.18 -0.25 ]


In [52]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# initial_motor = vsr.Vec(2,3,4).trs() * vsr.Rot(vsr.Biv(0,-1,0) * np.pi/6)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(circs_a, circs_b):
#     mes.add_circle_difference_residual_block(a,b)
    mes.add_circle_commutator_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary, motors2) = mes.solve()
print(summary['full_report'])
print(estimated_motor)


Solver Summary (v 1.12.0-eigen-(3.3.1)-lapack-suitesparse-(4.5.3)-cxsparse-(3.1.9)-no_openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            10                       10
Residual                                  100                      100

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1
Linear solver ordering              AUTOMATIC        

In [53]:
[np.allclose(m, m2,atol=1e-05) for m,m2 in zip(motors, motors2)]

[True, False, False, False, False, False]

In [54]:
print(zip(motors, motors2)[-1])

TypeError: 'zip' object is not subscriptable

In [55]:
scene = Scene(children=
#               [circle_mesh(c,color='hotpink') for c in circs_a] +\
#               [circle_mesh(c,color='lightskyblue') for c in circs_b] +\
              [circle_mesh(c.spin(motors[0]), color='red') for c in circs_a] +\
              [circle_mesh(c.spin(motors[1]), color='yellow') for c in circs_a] +\
              [circle_mesh(c.spin(motors2[1]), color='magenta') for c in circs_a] +\

#               [circle_mesh(c.spin(motors[2]), color='green') for c in circs_a] +\
# #               [circle_mesh(c.spin(motors[3]), color='cyan') for c in circs_a] +\
#               [circle_mesh(c.spin(motors[4]), color='blue') for c in circs_a] +\

#               [circle_mesh(c.spin(motors[-4]), color='green') for c in circs_a] +\
#               [circle_mesh(c.spin(motors2[-4]), color='red') for c in circs_a] +\
              [
        
                        frame_mesh(),
                        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)